In [1]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [3]:
#PET4

csv_pet4_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/data.csv'

with open(csv_pet4_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [8]:
csv_data[0]

['alina',
 '11011101021001',
 '0',
 '8',
 '1.2.840.113704.1.111.3296.1350645932.9',
 '2012-10-19',
 '0',
 '0',
 '0',
 'review8.csv',
 '4',
 '37.34',
 'Adrenal',
 '12.6',
 'Adrenal',
 '1.43',
 '1.17',
 '0.15',
 '2.56',
 '2.06',
 '0.18',
 '2.29',
 '1.64',
 '0.35',
 '0',
 '10.1',
 '2.13',
 '1.56',
 '0.2',
 '0',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Miss

In [7]:
csv_data[1][51]

'1.2.840.113704.1.111.4636.1364288605.3'

In [5]:
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'
liste_nifti = os.listdir(nifti_directory)

In [6]:
liste = []
for row in csv_data : 
    subliste = []
    #subliste.append(row[0])
    subliste.append(row[1])
    subliste.append(row[51])
    study_uid = row[51]
    subsubliste = []
    for nifti in liste_nifti : 
        if study_uid in nifti: 
            subsubliste.append(nifti)
    if len(subsubliste) == 2 : 
        subsubliste = sorted(subsubliste)
        #print(subsubliste)
        subliste.append(subsubliste[0])
        subliste.append(subsubliste[1])
        #subliste.append(subsubliste[2])
    else : 
        pass
    subliste.append("pet4")
    liste.append(subliste)



In [7]:
len(liste)

728

In [8]:
liste[10]

['11011101021011',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185_nifti_CT.nii',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185_nifti_PT.nii',
 'pet4']

In [9]:
#clean -> if len != 6 
#doit y avoir 573
completed_liste = []
for serie in liste : 
    if len(serie) == 5 : 
        completed_liste.append(serie)


In [10]:
len(completed_liste)

586

In [11]:
new_liste = []
new_liste.append(completed_liste[0])
for i in range(1, len(completed_liste)) :
    if completed_liste[i] not in new_liste : 
        new_liste.append(completed_liste[i])

In [12]:
len(new_liste)

527

In [24]:
new_liste[10]

['11011101021014',
 '1.2.840.113704.1.111.3608.1434978016.23',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_CT.nii',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_PT.nii',
 'pet2']

In [13]:
import shutil

In [16]:
new_liste[1]

['11011101021004',
 '1.2.840.113704.1.111.2304.1369728102.4',
 '1.2.840.113704.1.111.2304.1369728102.4_nifti_CT.nii',
 '1.2.840.113704.1.111.2304.1369728102.4_nifti_PT.nii',
 'pet4']

In [19]:
#creation sous dossier patient id -> pet0 -> mask ct pt 

for serie in new_liste: 
    patient_id = serie[0]
    pet_type = serie[-1]
    nifti_ct = serie[2]
    nifti_pet = serie[3]
        #nifti_mask = serie[5]

    folder = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI'+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "déjà crée => prblm")

    ct_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'+'/'+nifti_ct
    pt_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'+'/'+nifti_pet
        #mask_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'+'/'+nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct
    new_pet_path = subfolder+'/'+nifti_pet
        #new_mask_path = subfolder+'/'+nifti_mask
    try : 

        os.rename(ct_path, subfolder+'/'+nifti_ct)
        os.rename(pt_path, subfolder+'/'+nifti_pet)
        #os.rename(mask_path, subfolder+'/'+nifti_mask)
    except Exception as err : 
        print(err)


8024b95f92/GAINED_NIFTI/11011108371001 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371001/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371002 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371002/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371003 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371003/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371004 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371004/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108611001 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108611001/pet4 created
/

In [9]:


nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI'



In [10]:
csv_pet4_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/data.csv'

with open(csv_pet4_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [11]:
csv_data[0]


['alina',
 '11011101021001',
 '0',
 '8',
 '1.2.840.113704.1.111.3296.1350645932.9',
 '2012-10-19',
 '0',
 '0',
 '0',
 'review8.csv',
 '4',
 '37.34',
 'Adrenal',
 '12.6',
 'Adrenal',
 '1.43',
 '1.17',
 '0.15',
 '2.56',
 '2.06',
 '0.18',
 '2.29',
 '1.64',
 '0.35',
 '0',
 '10.1',
 '2.13',
 '1.56',
 '0.2',
 '0',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Missing Review',
 'Miss

In [19]:
data = []
for row in csv_data : 
    subliste = []
    subliste.append(row[1])
    subliste.append(row[51])
    subliste.append('pet4')
    if subliste[-2] != 'Missing Review' :
        data.append(subliste)

In [23]:
data[20]

['11011101031010',
 '1.2.840.113619.2.290.3.279710052.204.1392878330.491',
 'pet4']

In [24]:
for serie in data:
    folder = nifti_directory+'/'+serie[0]+'/'+'pet4'
    if os.path.exists(folder): 
        extract = os.listdir(folder)
        if 'CT' in extract[0] : 
            serie.append(folder+'/'+extract[0])
            serie.append(folder+'/'+extract[1])
        else : 
            serie.append(folder+'/'+extract[1])
            serie.append(folder+'/'+extract[0])



In [27]:
data[20]

['11011101031010',
 '1.2.840.113619.2.290.3.279710052.204.1392878330.491',
 'pet4',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011101031010/pet4/1.2.840.113619.2.290.3.279710052.204.1392878330.491_nifti_CT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011101031010/pet4/1.2.840.113619.2.290.3.279710052.204.1392878330.491_nifti_PT.nii']

In [31]:

cpt = 0
for serie in data : 
    if len(serie) != 5 : 
        print(serie)
        cpt += 1 
cpt

['11011101051007', '1.2.250.1.74.20130808091500.60000025600407', 'pet4']
['11011101051008', '1.2.250.1.74.20130823083500.60000025759284', 'pet4']
['11011101051009', '1.2.250.1.74.20131021075500.60000026411911', 'pet4']
['11011101051010', '1.2.250.1.74.20131021083500.60000026411852', 'pet4']
['11011101051011', '7.1712131130122501742013112007500060000026721932', 'pet4']
['51011102181002', '1.2.840.113619.2.55.3.279710477.550.1401430249.944', 'pet4']
['51011102181003', '1.2.840.113619.2.55.3.279710477.311.1408947299.335', 'pet4']
['51011102341001', '1.3.51.0.1.1.129.10.7.50.1308821.308851', 'pet4']
['51011102341002', '2.16.840.1.113669.632.20.330001.10002000027', 'pet4']
['51011102341003', '2.16.840.1.113669.632.20.330001.10002029541', 'pet4']
['51011102341004', '2.16.840.1.113669.632.20.330001.10002146308', 'pet4']
['51011102341005', '2.16.840.1.113669.632.20.330001.10002190354', 'pet4']
['51011102341006', '1.2.840.113704.1.111.7820.1387364903.1', 'pet4']
['51011102341007', '2.16.840.1.1

77

In [35]:
#doublon 
new_liste = []
new_liste.append(data[0])
for serie in data : 
    if serie not in new_liste and len(serie) == 5 : 
        new_liste.append(serie)


In [36]:
len(new_liste)

527

In [38]:
#write csv 
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI'

with open(os.path.join(nifti_directory, 'GAINED_PET4_NIFTI.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET"])
    for serie in new_liste: 
        if len(serie) == 5 : 
            csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4]])